In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-31388?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Machine Learning Engineer – AI/ML (Remote Work Option)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsMachine Learning Engineer – AI/ML (Remote Work Option)Be

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Machine Learning Engineer – AI/ML',
 'experience': 'Bachelor’s degree in a relevant field, like computer science or software engineering, or equivalent experience.',
 'skills': ['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
  'Ability to meaningfully communicate, written, orally, and visually technical topics with peers',
  'Experience working in and/or collaborating with a partial or fully distributed team',
  'Understanding of data structures, algorithms, and data solutions',
  'Experience in applying Python (or another language commonly used in the field of ML, such as Scala, Julia, C++) and SQL to ML and/or software and data engineering tasks',
  'Familiarity with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, or similar platforms and frameworks',
  'Awareness of data science platforms (like Databricks or SageMaker), distributed engines (like Spark and AWS cloud), a

In [16]:
job = json_res

In [17]:
type(json_res)

dict

In [18]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [19]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [21]:
job

{'role': 'Machine Learning Engineer – AI/ML',
 'experience': 'Bachelor’s degree in a relevant field, like computer science or software engineering, or equivalent experience.',
 'skills': ['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
  'Ability to meaningfully communicate, written, orally, and visually technical topics with peers',
  'Experience working in and/or collaborating with a partial or fully distributed team',
  'Understanding of data structures, algorithms, and data solutions',
  'Experience in applying Python (or another language commonly used in the field of ML, such as Scala, Julia, C++) and SQL to ML and/or software and data engineering tasks',
  'Familiarity with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, or similar platforms and frameworks',
  'Awareness of data science platforms (like Databricks or SageMaker), distributed engines (like Spark and AWS cloud), a

In [22]:
job = json_res
job['skills']

['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
 'Ability to meaningfully communicate, written, orally, and visually technical topics with peers',
 'Experience working in and/or collaborating with a partial or fully distributed team',
 'Understanding of data structures, algorithms, and data solutions',
 'Experience in applying Python (or another language commonly used in the field of ML, such as Scala, Julia, C++) and SQL to ML and/or software and data engineering tasks',
 'Familiarity with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, or similar platforms and frameworks',
 'Awareness of data science platforms (like Databricks or SageMaker), distributed engines (like Spark and AWS cloud), and CI/CD pipelines and containerization']

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Engineers for Your AI/ML Team

Dear Hiring Manager,

I came across your job posting for a Machine Learning Engineer – AI/ML, and I'm excited to introduce you to AtliQ, a leading AI & Software Consulting company. Our team of experts has extensive experience in developing advanced analytics systems that drive business growth.

At AtliQ, we understand the importance of seamless integration of business processes through automated tools. Our expertise in Machine Learning, data structures, algorithms, and data solutions aligns perfectly with your requirements. We have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team is well-versed in applying Python, SQL, and other relevant technologies to ML and software engineering tasks. We're familiar with ETL, ML, or analytics technologies such as Scikit-learn, Pandas, OpenCV, NumPy, TensorFlow, and similar p